In [13]:
import pandas as pd
from Bio import SeqIO, AlignIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, Gapped
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.SeqRecord import SeqRecord
import numpy as np

https://www.biostars.org/p/430724/

https://github.com/michaelgruenstaeudl/annonex2embl

https://biopython.org/docs/1.75/api/Bio.Seq.html

https://biopython.org/docs/1.75/api/Bio.SeqRecord.html

https://ena-docs.readthedocs.io/en/latest/submit/assembly/genome.html 

https://www.programcreek.com/python/example/114216/Bio.SeqFeature.SeqFeature

# Load samples

In [14]:
wdir = 'cpTree_v6'

In [15]:
# List of samples
cp_df = pd.read_csv(wdir + '/' + wdir + '_tree_clean.csv').drop(columns=['nentries','TaxID','mol_type','Label'])
print(cp_df.shape[0],cp_df.DataSource.value_counts().to_dict())
cp_df.head(2)

16682 {'REFSEQ': 6020, 'INSDC': 3948, 'PAFTOL': 3535, 'SRA': 1612, 'OneKP': 665, 'Lietal': 549, 'GAP': 343, 'Gymno': 10}


,Sample_Name,genus,species,sci_name,DataSource,length,order,class,family
0,GAP_026547,Albidella,oligococca,Albidella oligococca,GAP,59917.0,Alismatales,monocots,Alismataceae
1,GAP_026933,Stuckenia,pectinata,Stuckenia pectinata,GAP,31766.0,Alismatales,monocots,Potamogetonaceae


In [16]:
# Latest DB
db = pd.read_csv('C:/Data/PAFTOL/PAFTOL_DB/2022-01-31_paftol_export_stats.csv')
print(db.shape[0],db.columns)
# PAFTOL samples only
db_A353 = db[db.DataSource.isin(['GAP','PAFTOL'])]
db_A353['Sample_Name']= db_A353.DataSource + '_' + db_A353.idSequencing.astype(int).astype('str').str.zfill(6)
print(db_A353.shape[0])
db_A353.head(2)

17187 Index(['idSpecimen', 'idPaftol', 'DataSource', 'Project', 'Order', 'Family',
       'Genus', 'Species', 'Taxonomical_Notes', 'KewID', 'Collector',
       'CollectorNo', 'VoucherNo', 'BankID', 'MuseumID', 'MuseumName',
       'MuseumBarcode', 'HerbcatURL', 'LCD', 'MSB', 'SourceSpecimen',
       'country', 'country_code', 'SpecimenReference', 'Gymnosperm',
       'idSample', 'MaterialSource', 'AgeOfMaterial', 'Action',
       'Sample_Description', 'ExtractionType', 'DNAVolume', 'Quality',
       'SampleConcentration', 'NewSampleConcentration', 'GelImage',
       'SampleTapeStation', 'Compliant', 'SecENASampleNum', 'idLibrary',
       'GenerateLibrary', 'Sonication', 'LibConcentration', 'LibTapeStation',
       'Plate No', 'LibQuality', 'RemainingVolume', 'Library_Description',
       'Indexes', 'IndexNameFwd', 'SeqFwdPlatform1', 'SeqFwdPlatform2',
       'IndexNameRv', 'SeqRv', 'Library_Status', 'Coordinate', 'idSequencing',
       'ExternalSequenceID', 'HasDuplicate', 'IsMerged', 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (29,35,45,57,68,77) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\kle10kg\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,idSpecimen,idPaftol,DataSource,Project,Order,Family,Genus,Species,Taxonomical_Notes,KewID,...,sci_name,Sample,ENASampleNum,num_seqs,sum_len,min_len,max_len,NCBI_TaxID,NCBI_sciname,Sample_Name
0,1,1,PAFTOL,Asteraceae,Asterales,Asteraceae,Gymnolaena,sp.,Initially recorded as Gymnolaena litoralis but...,295689-2,...,Gymnolaena sp. - Jan-93 (K),564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAFTOL_000564
1,2,2,PAFTOL,Pilot,Ranunculales,Circaeasteraceae,Circaeaster,agrestis,NaN,167828-1,...,Circaeaster agrestis,961,ERS5501508,332.0,138795.0,72.0,"1,485",39288.0,Circaeaster agrestis,PAFTOL_000961


In [17]:
# Merge dataframes
## Q. Add voucher info?
cp_df = pd.merge(cp_df[['Sample_Name', 'family', 'genus', 'species', 'sci_name', 'DataSource']],
                  db_A353[['Sample_Name','idPaftol','KewID','idSequencing','ENASampleNum','ENAExpNumber','ENARunNumber','NCBI_TaxID','NCBI_sciname']],
                  how='left',on='Sample_Name')
print(cp_df.shape[0], cp_df.isna().sum().to_dict())
A353_df = cp_df[cp_df.ENARunNumber.notnull()]
print(A353_df.shape[0])
A353_df = A353_df[A353_df.NCBI_TaxID.notnull()].astype({'NCBI_TaxID':'int'})
print(A353_df.shape[0])
A353_df.head(2)

16682 {'Sample_Name': 0, 'family': 0, 'genus': 0, 'species': 0, 'sci_name': 0, 'DataSource': 0, 'idPaftol': 12804, 'KewID': 12812, 'idSequencing': 12804, 'ENASampleNum': 13413, 'ENAExpNumber': 13374, 'ENARunNumber': 13374, 'NCBI_TaxID': 12992, 'NCBI_sciname': 12992}
3308
3165


,Sample_Name,family,genus,species,sci_name,DataSource,idPaftol,KewID,idSequencing,ENASampleNum,ENAExpNumber,ENARunNumber,NCBI_TaxID,NCBI_sciname
0,GAP_026547,Alismataceae,Albidella,oligococca,Albidella oligococca,GAP,12214.0,77163628-1,26547.0,NaN,ERX7344886,ERR7776839,2890917,Albidella oligococca
1,GAP_026933,Potamogetonaceae,Stuckenia,pectinata,Stuckenia pectinata,GAP,12456.0,77099639-1,26933.0,NaN,ERX7170367,ERR7599787,55444,Stuckenia pectinata


In [18]:
#tmp 
removed_taxa = ['Murraya exotica']
A353_df = A353_df[A353_df.sci_name.isin(removed_taxa)==False]
print(A353_df.shape[0])
A353_df = A353_df[:200]
print(A353_df.shape[0])

3164
200


In [19]:
# List of genes
genes_df = pd.read_csv('cp_genes_all.csv')
print(genes_df.shape[0],genes_df.isna().sum().to_dict())
# tmp 
genes_df = genes_df[:4]
genes_df

79 {'Gene': 0, 'Product': 75, 'type': 75}


,Gene,Product,type
0,rbcL,"ribulose 1,5-bisphosphate carboxylase/oxygenas...",cds
1,rrn16,16S ribosomal RNA,rRNA
2,accD,AccD,cds
3,matK,matK,cds


# Processing Alignment and generation of EMBL file

In [20]:
def gzip_file(in_file):
    import gzip
    import shutil
    f_gz = in_file + '.gz'
    with open(in_file, 'rb') as f_in:
        with gzip.open(f_gz, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [23]:
def cut_to_codon(record):
    # Get index of first nucleotide
    pos_ini = len(record.seq) - len(record.seq.lstrip('-')) 
    # length of leading gap divisible by 3
    to_del = ((pos_ini // 3) * 3)
    # Update record without leading gap divisible by 3
    record.seq = record.seq[to_del:]
    # Update position of first nucleotide
    pos_ini = len(record.seq) - len(record.seq.lstrip('-'))
    # No change if first position
    if pos_ini==0:
        record.seq = record.seq
    # Cut partial codon otherwise
    elif pos_ini==1:
        record.seq = record.seq[3:]
    elif pos_ini==2: 
        record.seq = record.seq[3:]
    return record
    
rec_test = SeqRecord(Seq('--aatgca'), id='test_seq')
print(rec_test.seq)
new_rec = cut_to_codon(record=rec_test)
new_rec.seq

--aatgca


Seq('atgca')

In [10]:
def cut_seq(records,cut):
    for record in records:
        record.seq = record.seq[cut:]
    return records
# print(len(alignment[0].seq))
# tmp = cut_seq(records = alignment[:2], cut = 1)
# print(len(tmp[0].seq))

In [13]:
for idx, row in genes_df.iterrows():
    ## Load alignment
    alignment = AlignIO.read(open(wdir + '/Aligned_genes/' + row.Gene + '_aln.fasta'), "fasta", alphabet=Gapped(generic_dna))
    print(row.Gene, len(alignment))
    
    ## Read stats from aligned sequences
    record_ls = []; 
    for record in alignment:
        try:
            record_ls.append({'Sample_Name':record.id,'coverage':record.description.split('reference_coverage:')[1].split(',')[0],
                              'Leadgap':len(record.seq) - len(record.seq.lstrip('-')),'pident':record.description.split('pident:')[1].split(',')[0]})
        except:
            record_ls.append({'Sample_Name':record.id})
            
    ## Merge alignment df with samples df      
    record_df = pd.merge(cp_df,pd.DataFrame(record_ls).astype({'coverage':'float','pident':'float'}),how='left',on='Sample_Name')
    print('record_ls:',len(record_ls),'cp_df:',cp_df.shape[0],'record_df:',record_df.shape[0])
    
    ## find first refseq with 100% coverage
    print('coverage:',record_df.coverage.quantile([.1,.5,.9]).to_dict())
    ref100 = record_df[(record_df.DataSource=='REFSEQ') & (record_df.coverage==100.0)]
    print('Leadgap refseq:',ref100.Leadgap.quantile([.1,.5,.9]).to_dict())
    print('pident refseq:',ref100.pident.quantile([.1,.5,.9]).to_dict())
    
    ## Cut leading gap
    alignment = cut_seq(records = alignment, cut = int(ref100.Leadgap.median()))
    
    ## Create embl records
    record_export = []
    for record in alignment:
        if record.id in A353_df.Sample_Name.to_list():
            rec_sample_dc = record_df[record_df.Sample_Name==record.id].to_dict('records')[0]
            new_description = rec_sample_dc['NCBI_sciname'] + ' ' + row.Product
            if rec_sample_dc['coverage']==100:
                new_description = new_description + ', complete sequence'
            else:
                new_description += ', partial sequence'
            new_description += ', isolate ' + rec_sample_dc['ENARunNumber']
            rec = SeqRecord(record.seq, id=rec_sample_dc['ENARunNumber'] + '-' + row.Gene, name=rec_sample_dc['Sample_Name'],
                       description= new_description, dbxrefs=['taxon:' + str(int(rec_sample_dc['NCBI_TaxID']))]) #.ungap("-")
            rec = SeqRecord(cut_to_codon(record=record).seq.ungap('-'), id=rec_sample_dc['ENARunNumber'] + '-' + row.Gene, name=rec_sample_dc['Sample_Name'],
                       description= new_description, dbxrefs=['taxon:' + str(int(rec_sample_dc['NCBI_TaxID']))]) #.ungap("-")
            
            rec.annotations["molecule_type"]="genomic DNA";
            rec.annotations["topology"]="linear";
            rec.annotations["organism"]=rec_sample_dc['NCBI_sciname'];
            rec.annotations["taxon"]=str(int(rec_sample_dc['NCBI_TaxID']));
            rec.annotations["data_file_division"]="PLN";
            rec.features.append(SeqFeature(FeatureLocation(0, len(record.seq.ungap('-'))),
                                           type = "source", 
                                           qualifiers  = {'isolate':rec_sample_dc['ENARunNumber'],
                                                          'organism':rec_sample_dc['NCBI_sciname'],
                                                          'organelle':"plastid",
                                                          'mol_type':'genomic DNA',
                                                         'db_xref': ['taxon:' + str(int(rec_sample_dc['NCBI_TaxID']))] }
                                          ))

            rec.features.append(SeqFeature(FeatureLocation(0, len(record.seq.ungap('-'))),
                                           type = "gene", 
                                           qualifiers  = {'gene':row.Gene}))
            if row['type'] == 'cds':
                rec.features.append(SeqFeature(FeatureLocation(0, len(record.seq.ungap('-'))),
                                           type = "cds", 
                                           qualifiers  = {'gene':row.Gene,
                                                          'codon_start':1,
                                                          'transl_table':11,
                                                          'product':row.Product}))
            elif row['type'] == 'rRNA':
                rec.features.append(SeqFeature(FeatureLocation(0, len(record.seq.ungap('-'))),
                                           type = "rRNA", 
                                           qualifiers  = {'gene':row.Gene,
                                                          'product':row.Product}))
            record_export.append(rec)
    print('N records:',len(record_export))
    
    ## Write embl flat file, metadata table,  and submission commands
    SeqIO.write(record_export, wdir + '/ENA_gene_submission/' + row.Gene + '_A353.embl', "embl")
    
    ## Convert .gb to EMBL flat files
    gzip_file(wdir + '/ENA_gene_submission/' + row.Gene + '_A353.embl')
    
    ## Manifest file 
    manifest={
        'STUDY':'PRJEB35285', #https://www.ncbi.nlm.nih.gov/bioproject/588607
        'Name':row.Gene,
        'FLATFILE': row.Gene + '_A353.embl.gz',
        }
    manifest = pd.DataFrame.from_dict(manifest,orient='index')
    manifest.to_csv(wdir + '/ENA_gene_submission/' + row.Gene + '_A353_manifest.txt',header=None,sep='\t')

    ## Submission command
    print('java -jar C:/Data/PAFTOL/Organelles/ENA_submissions/webin-cli-4.3.0.jar -username Webin-52995' \
          + ' -passwordFile C:/Data/PAFTOL/Organelles/ENA_submissions/ena_pwd.txt -context sequence -manifest ' \
          + row.Gene + '_A353_manifest.txt' + ' -validate')
    print(' ')

rbcL 14847
record_ls: 14847 cp_df: 16682 record_df: 16682
coverage: {0.1: 97.8, 0.5: 99.9, 0.9: 100.0}
Leadgap refseq: {0.1: 0.0, 0.5: 0.0, 0.9: 0.0}
pident refseq: {0.1: 94.19, 0.5: 97.07, 0.9: 99.65}
N records: 136
java -jar C:/Data/PAFTOL/Organelles/ENA_submissions/webin-cli-4.3.0.jar -username Webin-52995 -passwordFile C:/Data/PAFTOL/Organelles/ENA_submissions/ena_pwd.txt -context sequence -manifest rbcL_A353_manifest.txt -validate
 
rrn16 15731
record_ls: 15731 cp_df: 16682 record_df: 16682
coverage: {0.1: 100.0, 0.5: 100.0, 0.9: 100.0}
Leadgap refseq: {0.1: 0.0, 0.5: 0.0, 0.9: 0.0}
pident refseq: {0.1: 99.19, 0.5: 99.87, 0.9: 100.0}
N records: 198
java -jar C:/Data/PAFTOL/Organelles/ENA_submissions/webin-cli-4.3.0.jar -username Webin-52995 -passwordFile C:/Data/PAFTOL/Organelles/ENA_submissions/ena_pwd.txt -context sequence -manifest rrn16_A353_manifest.txt -validate
 
accD 11135
record_ls: 11135 cp_df: 16682 record_df: 16682
coverage: {0.1: 69.7, 0.5: 99.9, 0.9: 100.0}
Leadgap r